In [ ]:
%pip install scikit-learn
%pip install matplotlib
%pip install umap-learn

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import os
import datetime
import csv
import json
import glob
from sklearn.decomposition import PCA
from umap import UMAP


In [ ]:
from functools import reduce


def complexity_reduce_tsne(data, dim):
    tsne = TSNE(n_components=dim, random_state=0)
    data_tsne = tsne.fit_transform(data)
    return data_tsne

def complexity_reduce_umap(data, dim):
    umap = UMAP(n_components=dim, random_state=0)
    data_umap = umap.fit_transform(data)
    return data_umap

def complexity_reduce_pca(data, dim):
    pca = PCA(n_components=dim, random_state=0)
    data_pca = pca.fit_transform(data)
    return data_pca

def similarity_from_complexity():
    save_dir_path = "./data/sims/complexity"
    os.makedirs(save_dir_path, exist_ok=True)

    data_path = "./data/_json/complexity/complexity_status.json"
    complexity_file_path = glob.glob(data_path)[0]
    data = None
    with open(complexity_file_path) as f:
        data = json.load(f)

    if data is None:
        print("complexity ta is None")
        return
    
    reduced_data = complexity_reduce_tsne(data, 8)